In [1]:
import numpy as np
import torch
import torchvision
from torchvision import datasets, transforms
import torch.nn as nn
from PIL import Image
import matplotlib.pyplot as plt

In [33]:
def get_acc_loss(model, loader):
    correct=0
    total=0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        output=model(images)
        pred = output.data.max(1, keepdim=True)[1]
        total+=labels.shape[0]
        correct += pred.eq(labels.data.view_as(pred)).sum()
        loss=loss_func(output, labels)
    acc=100.*correct/total
    return acc.detach().cpu().numpy(), loss.detach().cpu().numpy()

In [5]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])

trainset = torchvision.datasets.CIFAR10(root='./', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [7]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=True)

### CNN

In [78]:
class AnimalCNN(nn.Module):
    
    def __init__(self):
        super(AnimalCNN, self).__init__()
        self.flatten = nn.Flatten()
        self.convolution_stack = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3,3), stride=1, padding=1),   #in = 3x32x32; out = 32x32x32
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2)),      # in = 32x32x32:  out = 32x16x16
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3), stride=1, padding=1),   #in = 32x16x16; out = 64x16x16
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3), stride=1, padding=1),   #in = 32x16x16; out = 64x16x16
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2)),      # in = 64x16x16:  out = 64x8x8
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), stride=1, padding=1),   #in = 64x8x8; out = 128x8x8
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3,3), stride=1, padding=1),   #in = 64x8x8; out = 128x8x8
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2))      # in = 128x8x8:  out = 128x4x4
        )
        self.linear_stack=nn.Sequential(
            nn.Linear(128*4*4, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512,10)
        )

    def forward(self, x):
        x = self.convolution_stack(x)
        x=self.flatten(x)
        x=self.linear_stack(x)
        return x

In [79]:
model=AnimalCNN().to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9,0.999))

In [80]:
n_epochs=5
train_loss = []
test_loss = []
train_acc=[]
test_acc=[]

In [81]:
%%time

for epoch in range(n_epochs):
    model.train()
    for idx, (images, labels) in enumerate(trainloader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(images)
        loss=loss_func(output, labels)
        loss.backward()
        optimizer.step()


    model.eval()
    with torch.inference_mode():
        acc, loss = get_acc_loss(model, trainloader)
        train_acc.append(acc)
        train_loss.append(loss)
        
        acc, loss = get_acc_loss(model, testloader)            
        test_acc.append(acc)
        test_loss.append(loss)


  #  if epoch%10==9:
    print(f'Epoch {epoch+1}/{n_epochs}, Test Loss={loss.item():.3}, Test Accuracy={acc:.3}')

Epoch 1/5, Test Loss=0.685, Test Accuracy=60.0
Epoch 2/5, Test Loss=0.742, Test Accuracy=67.4
Epoch 3/5, Test Loss=1.83, Test Accuracy=64.5
Epoch 4/5, Test Loss=1.17, Test Accuracy=74.7
Epoch 5/5, Test Loss=0.884, Test Accuracy=77.7
CPU times: total: 1min 52s
Wall time: 1min 28s


In [73]:
class AnimalResNet(nn.Module):
    
    def __init__(self):
        super(AnimalResNet, self).__init__()
        self.flatten = nn.Flatten()
        self.first_convolution_stack = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(3,3), stride=1, padding=1),   #in = 3x32x32; out = 64x32x32
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2))      # in = 64x32x32:  out = 64x16x16
        )
        self.res_convolution_stack1 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3), stride=1, padding=1),   #in = 64x16x16; out = 64x16x16
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3), stride=1, padding=1),   #in = 64x16x16; out = 64x16x16
            nn.BatchNorm2d(64)
        )
        self.bottleneck1=nn.Sequential(
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), stride=1, padding=1),   #in = 64x16x16; out = 128x16x16
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=(2,2))  #in = 128x16x16; out = 128x8x8
        )
            
        self.res_convolution_stack2 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3,3), stride=1, padding=1),   #in = 128x8x8; out = 128x18x8
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3,3), stride=1, padding=1),   #in = 128x8x8; out = 128x8x8
            nn.BatchNorm2d(128)
        )
        self.bottleneck2=nn.Sequential(
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3,3), stride=1, padding=1),   #in = 128x8x8; out = 128x8x8
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=(2,2))  #in = 128x8x8; out = 128x4x4
        )
        self.linear_stack=nn.Sequential(
            nn.Linear(128*4*4, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512,10)
        )

    def forward(self, x):
       
        x = self.first_convolution_stack(x)
        save=x 
        x=self.res_convolution_stack1(x)
        x=x+save
        x=self.bottleneck1(x)
        save=x
        x=self.res_convolution_stack2(x)
        x=x+save
        x=self.bottleneck2(x)

        x=self.flatten(x)

        x=self.linear_stack(x)
        return x

In [66]:
model2=AnimalResNet().to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.001, betas=(0.9,0.999))

In [67]:
n_epochs=5
train_loss = []
test_loss = []
train_acc=[]
test_acc=[]

In [68]:
%%time

for epoch in range(n_epochs):
    model2.train()
    for idx, (images, labels) in enumerate(trainloader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model2(images)
        loss=loss_func(output, labels)
        loss.backward()
        optimizer.step()


    model2.eval()
    with torch.inference_mode():
        acc, loss = get_acc_loss(model2, trainloader)
        train_acc.append(acc)
        train_loss.append(loss)
        
        acc, loss = get_acc_loss(model2, testloader)            
        test_acc.append(acc)
        test_loss.append(loss)


  #  if epoch%10==9:
    print(f'Epoch {epoch+1}/{n_epochs}, Test Loss={loss.item():.3}, Test Accuracy={acc:.3}')

Epoch 1/5, Test Loss=1.17, Test Accuracy=65.1
Epoch 2/5, Test Loss=1.39, Test Accuracy=71.9
Epoch 3/5, Test Loss=0.409, Test Accuracy=75.8
Epoch 4/5, Test Loss=0.421, Test Accuracy=77.9
Epoch 5/5, Test Loss=0.258, Test Accuracy=80.2
CPU times: total: 1min 50s
Wall time: 1min 27s
